In [1]:
import duckdb
from pipelines.tasks._common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [2]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [3]:
%sql select distinct de_partition from edc_resultats;

# petite vérification des années importées dans la BDD

Running query in 'duckdb:///../../database/data.duckdb'

de_partition
2022
2023
2024
2020
2021


## Première solution

On compte le nombre de fois que chaque paramètre est apparu sur les années 2020-2024. Pour cela, on garde l'unicité sur ces colonnes:

- cdparametresiseeaux : Code SISE-Eaux du paramètre
- cdparametre : Code SANDRE du paramètre
- libmajparametre : Nom du paramètre en majuscule
- libminparametre : Nom du paramètre en minuscule
- casparam : Code CAS du paramètre

Et un fait un simple compte.


In [4]:
%%sql result1 <<
SELECT
    cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam, COUNT(*) as count
FROM edc_resultats
GROUP BY cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam
ORDER BY count DESC;

Running query in 'duckdb:///../../database/data.duckdb'

In [5]:
result1  # noqa: F821

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,count
PH,1302,PH,pH,None,1742021
CDT25,1303,CONDUCTIVITÉ À 25°C,Conductivité à 25°C,None,1367240
TEAU,1301,TEMPÉRATURE DE L'EAU,Température de l'eau,None,1334050
STRF,6455,ENTÉROCOQUES /100ML-MS,Entérocoques /100ml-MS,None,1316301
ECOLI,1449,ESCHERICHIA COLI /100ML - MF,Escherichia coli /100ml - MF,None,1316287
CTF,1447,BACTÉRIES COLIFORMES /100ML-MS,Bactéries coliformes /100ml-MS,None,1315825
TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,1313954
CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,1310664
GT36_44,5441,BACT. AÉR. REVIVIFIABLES À 36°-44H,Bact. aér. revivifiables à 36°-44h,None,1308998
GT22_68,5440,BACT. AÉR. REVIVIFIABLES À 22°-68H,Bact. aér. revivifiables à 22°-68h,None,1306806


Et maintenant la même chose mais on distingue année par année. Peut être utile pour voir l'évolution des paramètres.


In [6]:
%%sql result2 <<
SELECT
    de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam, COUNT(*) as count
FROM edc_resultats
GROUP BY de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam
ORDER BY de_partition, count DESC;

Running query in 'duckdb:///../../database/data.duckdb'

In [7]:
result2  # noqa: F821

de_partition,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,count
2020,PH,1302,PH,pH,None,338110
2020,CDT25,1303,CONDUCTIVITÉ À 25°C,Conductivité à 25°C,None,263505
2020,TEAU,1301,TEMPÉRATURE DE L'EAU,Température de l'eau,None,262440
2020,ECOLI,1449,ESCHERICHIA COLI /100ML - MF,Escherichia coli /100ml - MF,None,257230
2020,STRF,6455,ENTÉROCOQUES /100ML-MS,Entérocoques /100ml-MS,None,257229
2020,CTF,1447,BACTÉRIES COLIFORMES /100ML-MS,Bactéries coliformes /100ml-MS,None,257158
2020,TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,256959
2020,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,254069
2020,GT36_44,5441,BACT. AÉR. REVIVIFIABLES À 36°-44H,Bact. aér. revivifiables à 36°-44h,None,253182
2020,CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,253105


## Deuxième solution

On va rajouter le nombre de communes distincts sur lesquels chaque paramètre est apparu.

On va devoir faire une jointure entre la table `edc_resultats` et la table `edc_prelevements` pour récupérer la colonne `inseecommuneprinc`. Or, la relation entre ces deux tables semble N-N (du moins, 1 `referenceprel` dans `edc_resultats` peut correspondre à plusieurs `referenceprel` dans `edc_prelevements`). On va donc aggréger les données de `edc_prelevements` avant la jointure et faire des comptages pour vérifier le join.


In [8]:
%%sql

select count(*) from (select * from edc_resultats)

# on compte le nombre de lignes dans la tabble ; on veut le même nombre après notre join

Running query in 'duckdb:///../../database/data.duckdb'

count_star()
64164498


In [9]:
%%sql

select count(*) from (
    with prelevements_unique as (
        select
            de_partition,
            referenceprel,
            inseecommuneprinc
        from edc_prelevements
        group by de_partition, referenceprel, inseecommuneprinc
    )
    select
        *
    from edc_resultats
    left join prelevements_unique on
        edc_resultats.referenceprel = prelevements_unique.referenceprel
        and edc_resultats.de_partition = prelevements_unique.de_partition
)

# on fait un group by et un join, on vériie que le nombre de lignes est le même

Running query in 'duckdb:///../../database/data.duckdb'

count_star()
64164498


In [10]:
%%sql

with prelevements_unique as (
    select
        de_partition,
        referenceprel,
        inseecommuneprinc
    from edc_prelevements
    group by de_partition, referenceprel, inseecommuneprinc
)
select
    edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam, COUNT(*) as count, COUNT(DISTINCT inseecommuneprinc) as count_communes
from edc_resultats
left join prelevements_unique on
    edc_resultats.referenceprel = prelevements_unique.referenceprel
    and edc_resultats.de_partition = prelevements_unique.de_partition
GROUP BY edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam
ORDER BY edc_resultats.de_partition, count DESC;

# maintenant on peut faire nos aggrégations et compter le nombre de communes en réutilisant la même jointure

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

de_partition,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,count,count_communes
2020,PH,1302,PH,pH,None,338110,34097
2020,CDT25,1303,CONDUCTIVITÉ À 25°C,Conductivité à 25°C,None,263505,34101
2020,TEAU,1301,TEMPÉRATURE DE L'EAU,Température de l'eau,None,262440,34027
2020,ECOLI,1449,ESCHERICHIA COLI /100ML - MF,Escherichia coli /100ml - MF,None,257230,34098
2020,STRF,6455,ENTÉROCOQUES /100ML-MS,Entérocoques /100ml-MS,None,257229,34097
2020,CTF,1447,BACTÉRIES COLIFORMES /100ML-MS,Bactéries coliformes /100ml-MS,None,257158,34098
2020,TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,256959,34098
2020,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,254069,33586
2020,GT36_44,5441,BACT. AÉR. REVIVIFIABLES À 36°-44H,Bact. aér. revivifiables à 36°-44h,None,253182,33647
2020,CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,253105,33181


In [11]:
result3_df = con.query("""
with prelevements_unique as (
    select
        de_partition,
        referenceprel,
        inseecommuneprinc
    from edc_prelevements
    group by de_partition, referenceprel, inseecommuneprinc
)
select
    edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam, COUNT(*) as count, COUNT(DISTINCT inseecommuneprinc) as count_communes
from edc_resultats
left join prelevements_unique on
    edc_resultats.referenceprel = prelevements_unique.referenceprel
    and edc_resultats.de_partition = prelevements_unique.de_partition
GROUP BY edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam
ORDER BY edc_resultats.de_partition, count DESC;
""").to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Génération du Excel de sortie


In [12]:
# on crée un fichier excel avec 3 onglets, un onglet par résultat

import pandas as pd

result1_df = result1.DataFrame()  # noqa: F821
result2_df = result2.DataFrame()  # noqa: F821
# result3_df = result3.DataFrame()

with pd.ExcelWriter("output.xlsx", engine="xlsxwriter") as writer:
    result1_df.to_excel(writer, sheet_name="Sheet1")
    result2_df.to_excel(writer, sheet_name="Sheet2")
    result3_df.to_excel(writer, sheet_name="Sheet3")